In [ ]:
import os
import random
import smtplib
from email.mime.text import MIMEText
import re

# Set environment variables 
os.environ['SENDER_EMAIL'] = 'sitaramnsr1997@gmail.com'
os.environ['EMAIL_PASSWORD'] = 'cgrt ejfr ncgv pcly'

def generate_otp():
    """Generate a random 6-digit OTP."""
    return str(random.randint(100000, 999999))

def validate_email(email):
    """Validate the email address using a regex pattern."""
    email_regex = r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
    return re.match(email_regex, email)

def send_otp_email(otp, email):
    """Send the OTP to the user's email address."""
    sender_email = os.getenv('SENDER_EMAIL')
    password = os.getenv('EMAIL_PASSWORD')

    if not sender_email or not password:
        print("Email credentials are not set properly.")
        return False

    message = MIMEText(f"Your OTP is {otp}")
    message['Subject'] = 'Your OTP Code'
    message['From'] = sender_email
    message['To'] = email

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(sender_email, password)
            server.sendmail(sender_email, email, message.as_string())
        print(f"OTP sent to {email}")
        return True
    except smtplib.SMTPAuthenticationError:
        print("Failed to authenticate with the email server. Please check your credentials.")
    except smtplib.SMTPConnectError:
        print("Failed to connect to the email server. Please check your internet connection.")
    except Exception as e:
        print(f"Failed to send email. Error: {e}")

    return False

def request_new_otp():
    """Ask the user if they want to request a new OTP."""
    while True:
        retry = input("Would you like to request a new OTP? (yes/no): ").strip().lower()
        if retry in ['yes', 'y']:
            return True
        elif retry in ['no', 'n']:
            return False
        else:
            print("Please answer with 'yes' or 'no'.")

def main():
    while True:
        email = input("Enter your email address: ").strip()

        if not validate_email(email):
            print("Invalid email format. Please try again.")
            continue

        otp = generate_otp()
        if send_otp_email(otp, email):
            attempts = 3
            while attempts > 0:
                entered_otp = input("Please enter the OTP sent to your email: ").strip()
                if entered_otp == otp:
                    print("OTP Verified Successfully!")
                    return
                else:
                    attempts -= 1
                    print(f"Invalid OTP. You have {attempts} attempts left.")
                    if attempts == 0:
                        print("You have exhausted all attempts.")
                        if request_new_otp():
                            break
                        else:
                            print("Exiting the program.")
                            return
        else:
            print("Failed to send OTP. Please check your email address and try again.")

if __name__ == "__main__":
    main()
